## ROOT dataframe tutorial: Dimuon spectrum

This tutorial shows you how to analyze datasets using RDataFrame from a Python notebook. The example analysis performs the following steps:

* Connect a ROOT dataframe to a dataset containing 61 mio. events recorded by CMS in 2012
* Filter the events being relevant for your analysis
* Compute the invariant mass of the selected dimuon candidates
* Plot the invariant mass spectrum showing resonances up to the Z mass

This material is based on the analysis done by Stefan Wunsch, available [here](http://opendata.web.cern.ch/record/12342) in CERN's Open Data portal.

<center><img src="../../../images/dimuonSpectrum.png"></center>

In [1]:
%%time
import ROOT

CPU times: user 759 ms, sys: 217 ms, total: 975 ms
Wall time: 1.42 s


## Create a ROOT dataframe in Python
First we will create a ROOT dataframe that is connected to a dataset named `Events` stored in a ROOT file. The file is pulled in via [XRootD](http://xrootd.org/) from EOS public, but note how it could also be stored in your CERNBox space or in any other EOS repository accessible from SWAN (e.g. the experiment ones).

The dataset Events is a TTree and has the following branches:

| Branch name | Data type | Description |
|-------------|-----------|-------------|
| `nMuon` | `unsigned int` | Number of muons in this event |
| `Muon_pt` | `float[nMuon]` | Transverse momentum of the muons stored as an array of size `nMuon` |
| `Muon_eta` | `float[nMuon]` | Pseudo-rapidity of the muons stored as an array of size `nMuon` |
| `Muon_phi` | `float[nMuon]` | Azimuth of the muons stored as an array of size `nMuon` |
| `Muon_charge` | `int[nMuon]` | Charge of the muons stored as an array of size `nMuon` and either -1 or 1 |
| `Muon_mass` | `float[nMuon]` | Mass of the muons stored as an array of size `nMuon` |

In [2]:
%%time

treename = "Events"
filename = "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"
df = ROOT.RDataFrame(treename, filename)

df_range = df.Range(0, int(1e6))

CPU times: user 351 ms, sys: 36.5 ms, total: 388 ms
Wall time: 798 ms


In [ ]:
%%time

with ROOT.TFile.Open("root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root") as file:
    tree = file.Get("Events")
    tree.Scan("*")

In [ ]:
%%time

input_file = "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"
tree_name = "Events"
output_file = "RNTuple_TEST_Run2012BC_DoubleMuParked_Muons.root"



ROOT.EnableImplicitMT()
ROOT.ConvertTTreeToRNTuple(input_file, tree_name, output_file)
ROOT.DisableImplicitMT()
with ROOT.Experimental.RNTupleReader.Open("Events", "RNTuple_TEST_Run2012BC_DoubleMuParked_Muons.root") as reader:
    reader.PrintInfo()

## Run only on a part of the dataset

The full dataset contains half a year of CMS data taking in 2012 with 61 mio events. For the purpose of this example, we use the [Range](https://root.cern/doc/master/classROOT_1_1RDF_1_1RInterface.html#a1b36b7868831de2375e061bb06cfc225) node to run only on a small part of the dataset. This feature also comes in handy in the development phase of your analysis.

Feel free to experiment with this parameter!

In [3]:
# Take only the first 1M events
#df_range = df.Range(0, int(1e6))

In [4]:
#df_range.Count().GetValue()

## Filter relevant events for this analysis

Physics datasets are often general purpose datasets and therefore need extensive filtering of the events for the actual analysis. Here, we implement only a simple selection based on the number of muons and the charge to cut down the dataset in events that are relevant for our study.

In particular, we are applying two filters to keep:
1. Events with exactly two muons
2. Events with muons of opposite charge

In [5]:
# Change the first strings of both following operations to proper C++ expressions
# Use the points 1, 2 above as hints for what to write in your expression
#df_2mu = df_range.Filter("nMuon == 2", "Events with exactly two muons")
#df_oc = df_2mu.Filter("Muon_charge[0] != Muon_charge[1]", "Muons with opposite charge")

## Perform complex operations in Python, efficiently!

Operations in the RDataFrame event loop are executed in C++ to ensure performance and allow for multithreading scalability. In many cases, the functions needed for the analysis can be already found in the standard C++ library, in the ROOT library or in your favourite analysis framework. Here, we use a `Define` node to compute the invariant mass of the muons in the dataset. An implementation of this function is already available in the [`ROOT::VecOps`](https://root.cern/doc/master/group__vecops.html) namespace.

In [6]:
#df_mass = df_oc.Define("Dimuon_mass", "ROOT::VecOps::InvariantMass(Muon_pt, Muon_eta, Muon_phi, Muon_mass)")

## Make a histogram of the newly created column

In [7]:
# These are the parameters you would give to a histogram object constructor
# Put them in the right order inside the parentheses below
# You are effectively passing a tuple to the `Histo1D` operation as seen previously in other notebooks
#nbins = 30000
#low = 0.25
#up = 300
#histo_name = "Dimuon_mass"
#histo_title = histo_name

#h = df_mass.Histo1D((histo_name, histo_title, nbins, low, up), "Dimuon_mass")

## Book a Report of the dataframe filters

In [8]:
report = df.Report()

## Start data processing
This is the final step of the analysis: retrieving the result. We are expecting to see a plot of the mass of the dimuon spectrum similar to the one shown at the beginning of this exercise (remember we are running on fewer entries in this exercise). Finally in the last cell we should see a report of the filters applied on the dataset.

In [9]:
%%time

df_range = df.Range(0, int(1e6))

df_2mu = df_range.Filter("nMuon == 2", "Events with exactly two muons")
df_oc = df_2mu.Filter("Muon_charge[0] != Muon_charge[1]", "Muons with opposite charge")

df_mass = df_oc.Define("Dimuon_mass", "ROOT::VecOps::InvariantMass(Muon_pt, Muon_eta, Muon_phi, Muon_mass)")

nbins = 30000
low = 0.25
up = 300
histo_name = "Dimuon_mass"
histo_title = histo_name

h = df_mass.Histo1D((histo_name, histo_title, nbins, low, up), "Dimuon_mass")

ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetTextFont(42)
c = ROOT.TCanvas("c", "", 800, 700)
c.SetLogx()
c.SetLogy()
h.SetTitle("")
h.GetXaxis().SetTitle("m_{#mu#mu} (GeV)")
h.GetXaxis().SetTitleSize(0.04)
h.GetYaxis().SetTitle("N_{Events}")
h.GetYaxis().SetTitleSize(0.04)
h.Draw()

label = ROOT.TLatex()
label.SetNDC(True)
label.SetTextSize(0.040)
label.DrawLatex(0.100, 0.920, "#bf{CMS Open Data}")
label.SetTextSize(0.030)
label.DrawLatex(0.500, 0.920, "#sqrt{s} = 8 TeV, L_{int} = 11.6 fb^{-1}")

CPU times: user 7.81 s, sys: 110 ms, total: 7.92 s
Wall time: 8.48 s


In [10]:
%jsroot on
c.Draw()

In [11]:
report.Print()

Events with exactly two muons: pass=489473     all=1000000    -- eff=48.95 % cumulative eff=48.95 %
Muons with opposite charge: pass=371508     all=489473     -- eff=75.90 % cumulative eff=37.15 %


## Additional: store all your custom function in a separate header file

In addition, it is possible to store user-defined functions (like the invariant mass shown before) in a separate .h file. In this way, we can keep the notebook only for Dataframe operations.

To achieve this, **open** the .h file `rdataframe-dimuon.h` in the same folder of this notebook, and edit it to perform the same operation of the previous section.\
If you have doubts, look here in the [docs](https://root.cern.ch/doc/master/group__vecops.html#gaa5798925785053643e12a326044fab37) for the invariant mass definition!

In [12]:
#EDIT the .h file

Now you are ready to load it in the notebook execution with this command:

In [13]:
#def my_initialization_function():
#    ROOT.gInterpreter.Declare("#include \"rdataframe-dimuon.h\"")

#my_initialization_function()

And finally, define a new column with the custom function:

In [14]:
#df_mass_custom = df_oc.Define("Dimuon_mass_custom", "custom_InvariantMass(Muon_pt, Muon_eta, Muon_phi, Muon_mass)")

Now we can repeate the steps before, to see if the results are identical

In [15]:
#nbins = 30000
#low = 0.25
#up = 300
#histo_name = "Dimuon_mass_custom"
#histo_title = histo_name

#h_custom = df_mass_custom.Histo1D((histo_name, histo_title, nbins, low, up), "Dimuon_mass_custom")

In [16]:
#%%time

#ROOT.gStyle.SetOptStat(0)
#ROOT.gStyle.SetTextFont(42)
#c_custom = ROOT.TCanvas("c_custom", "", 800, 700)
#c_custom.SetLogx()
#c_custom.SetLogy()
#h_custom.SetTitle("")
#h_custom.GetXaxis().SetTitle("m_{#mu#mu} (GeV)")
#h_custom.GetXaxis().SetTitleSize(0.04)
#h_custom.GetYaxis().SetTitle("N_{Events}")
#h_custom.GetYaxis().SetTitleSize(0.04)
#h_custom.Draw()

#label = ROOT.TLatex()
#label.SetNDC(True)
#label.SetTextSize(0.040)
#label.DrawLatex(0.100, 0.920, "#bf{CMS Open Data}")
#label.SetTextSize(0.030)
#label.DrawLatex(0.500, 0.920, "#sqrt{s} = 8 TeV, L_{int} = 11.6 fb^{-1}")

In [17]:
#%jsroot on
#c_custom.Draw()

## Additional: run the example on the `KubeCluster`

We can also run this example on the `KubeCluster`.

- Create a new `KubeCluster`;
- Scale it with a few workers (a couple will be enough);
- Connect to the cluster, using with the `Client` object;
- Scale the cluster with 2-3 workers.

In [18]:
from dask.distributed import get_worker
from pathlib import Path

In [19]:
from dask.distributed import Client

client = Client("localhost:23932")
client

<Client: 'tcp://10.0.5.71:23932' processes=2 threads=2, memory=4.00 GiB>

After creating and scaling the cluster, you can simply copy the code above! 

The only differences:
- Remember to use `ROOT.RDF.Experimental.Distributed.Dask.RDataFrame` instead of `ROOT.RDataFrame`;
- You cannot use the `Range` method (yet!). However, now you are running the full dataset, with ~62M events!
- If you want to use custom functions, stored in a header file, load them in the notebook with the `ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)`

In [20]:
%%time

treename = "Events"
filename = "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"
df_dask = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame(treename, filename, npartitions=4, daskclient=client)

# We need to register the header file for upload to the Dask workers.
# For now the interface is still WIP, will be made smoother in the next ROOT release
df_dask._headnode.backend.distribute_unique_paths(
    ["rdataframe-dimuon.h"]
)

def my_initialization_function():
    """Load C++ helper functions. Works for both local and distributed execution."""
    try:
        # when using distributed RDataFrame 'rdataframe-dimuon.h' is copied to the local_directory
        # of every worker (via `distribute_unique_paths`)
        localdir = get_worker().local_directory
        cpp_header = Path(localdir) / "rdataframe-dimuon.h"
    except ValueError:
        # must be local execution
        cpp_header = "rdataframe-dimuon.h"

    ROOT.gInterpreter.Declare(f'#include "{str(cpp_header)}"')

ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)

CPU times: user 141 ms, sys: 22.6 ms, total: 164 ms
Wall time: 444 ms


In [21]:
#ROOT.gStyle.SetOptStat(0)
#ROOT.gStyle.SetTextFont(42)
#c = ROOT.TCanvas("c_dask", "", 800, 700)
#c.SetLogx()
#c.SetLogy()
#h.SetTitle("")
#h.GetXaxis().SetTitle("m_{#mu#mu} (GeV)")
#h.GetXaxis().SetTitleSize(0.04)
#h.GetYaxis().SetTitle("N_{Events}")
#h.GetYaxis().SetTitleSize(0.04)
#h.Draw()

#label = ROOT.TLatex()
#label.SetNDC(True)
#label.SetTextSize(0.040)
#label.DrawLatex(0.100, 0.920, "#bf{CMS Open Data}")
#label.SetTextSize(0.030)
#label.DrawLatex(0.500, 0.920, "#sqrt{s} = 8 TeV, L_{int} = 11.6 fb^{-1}")

In [22]:
#%jsroot on
#c.Draw()

In [23]:
%%time

df_2mu = df_dask.Filter("nMuon == 2", "Events with exactly two muons")
df_oc = df_2mu.Filter("Muon_charge[0] != Muon_charge[1]", "Muons with opposite charge")

#df_mass = df_oc.Define("Dimuon_mass", "ROOT::VecOps::InvariantMass(Muon_pt, Muon_eta, Muon_phi, Muon_mass)")
df_mass_custom = df_oc.Define("Dimuon_mass_custom", "custom_InvariantMass(Muon_pt, Muon_eta, Muon_phi, Muon_mass)")

nbins = 30000
low = 0.25
up = 300
#histo_name = "Dimuon_mass"
#histo_title = histo_name

histo_name_custom = "Dimuon_mass_custom"
histo_title_custom = histo_name

#h = df_mass.Histo1D((histo_name, histo_title, nbins, low, up), "Dimuon_mass")
h_custom = df_mass_custom.Histo1D((histo_name_custom, histo_title_custom, nbins, low, up), "Dimuon_mass_custom")

ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetTextFont(42)
c_custom = ROOT.TCanvas("c_dask_custom", "", 800, 700)
c_custom.SetLogx()
c_custom.SetLogy()
h_custom.SetTitle("")
h_custom.GetXaxis().SetTitle("m_{#mu#mu} (GeV)")
h_custom.GetXaxis().SetTitleSize(0.04)
h_custom.GetYaxis().SetTitle("N_{Events}")
h_custom.GetYaxis().SetTitleSize(0.04)
h_custom.Draw()

label = ROOT.TLatex()
label.SetNDC(True)
label.SetTextSize(0.040)
label.DrawLatex(0.100, 0.920, "#bf{CMS Open Data}")
label.SetTextSize(0.030)
label.DrawLatex(0.500, 0.920, "#sqrt{s} = 8 TeV, L_{int} = 11.6 fb^{-1}")

CPU times: user 188 ms, sys: 29.4 ms, total: 217 ms
Wall time: 45.8 s


In [24]:
%jsroot on
c_custom.Draw()

## TTree -> RNTuple conversion

In [25]:
import ROOT

ROOT.gInterpreter.ProcessLine("gInterpreter->Reset()")

# Load the C++ function and header file in ROOT
ROOT.gInterpreter.Declare("""
#include <ROOT/RNTupleDS.hxx>
#include <ROOT/RNTupleImporter.hxx>
#include <ROOT/RNTupleReader.hxx>
#include <ROOT/RPageStorageFile.hxx>
#include <TFile.h>
#include <TROOT.h>
#include <TSystem.h>
#include <iostream>

void ConvertTTreeToRNTuple(const std::string &inputFileName, const std::string &treeName,
                           const std::string &outputFileName, const std::string &ntupleName = "Events") {
    using RNTupleImporter = ROOT::Experimental::RNTupleImporter;
    using RNTupleReader = ROOT::Experimental::RNTupleReader;

    gSystem->Unlink(outputFileName.c_str());

    auto importer = RNTupleImporter::Create(inputFileName, treeName, outputFileName);
    if (!importer) {
        std::cerr << "Failed to create RNTupleImporter!" << std::endl;
        return;
    }
    importer->Import();

    std::cout << "RNTuple created successfully in " << outputFileName << std::endl;
}
""")


True

In [26]:
%%time

input_file = "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"
tree_name = "Events"
output_file = "RNTuple_TEST_Run2012BC_DoubleMuParked_Muons.root"

ROOT.EnableImplicitMT()
ROOT.ConvertTTreeToRNTuple(input_file, tree_name, output_file)
ROOT.DisableImplicitMT()

CPU times: user 1min 35s, sys: 59.7 s, total: 2min 35s
Wall time: 2min 26s
Importing 'Muon_pt' [float]
Importing 'Muon_eta' [float]
Importing 'Muon_phi' [float]
Importing 'Muon_mass' [float]
Importing 'Muon_charge' [std::int32_t]
Wrote 95MB, 3785892 entries
Wrote 145MB, 5763181 entries
Wrote 195MB, 7739089 entries
Wrote 245MB, 9715856 entries
Wrote 295MB, 11695408 entries
Wrote 345MB, 13672188 entries
Wrote 395MB, 15651787 entries
Wrote 445MB, 17629342 entries
Wrote 495MB, 19607515 entries
Wrote 545MB, 21585884 entries
Wrote 595MB, 23560703 entries
Wrote 645MB, 25536552 entries
Wrote 695MB, 27459671 entries
Wrote 746MB, 29359352 entries
Wrote 796MB, 31258219 entries
Wrote 846MB, 33158598 entries
Wrote 896MB, 35055695 entries
Wrote 947MB, 36955395 entries
Wrote 997MB, 38856258 entries
Wrote 1047MB, 40750399 entries
Wrote 1097MB, 42652330 entries
Wrote 1147MB, 44551827 entries
Wrote 1197MB, 46450272 entries
Wrote 1248MB, 48350527 entries
Wrote 1298MB, 50249664 entries
Wrote 1348MB, 52141

Warning in <[ROOT.NTuple] Warning /home/conda/feedstock_root/build_artifacts/root_base_1720427374106/work/root-source/tree/ntuple/v7/src/RPageStorageFile.cxx:56 in ROOT::Experimental::Internal::RPageSinkFile::RPageSinkFile(std::string_view, const ROOT::Experimental::RNTupleWriteOptions&)::<lambda()>>: The RNTuple file format will change. Do not store real data with this version of RNTuple!


In [27]:
df_rnt = ROOT.RDataFrame(treename, output_file)

Warning in <[ROOT.NTuple] Warning /home/conda/feedstock_root/build_artifacts/root_base_1720427374106/work/root-source/tree/ntuple/v7/src/RPageStorageFile.cxx:262 in ROOT::Experimental::Internal::RPageSourceFile::InitDescriptor(const ROOT::Experimental::RNTuple&)::<lambda()>>: Pre-release format version: RC 2


In [28]:
%%time
# Take only the first 1M events
df_range = df_rnt.Range(0, int(1e6))
df_2mu = df_range.Filter("nMuon == 2", "Events with exactly two muons")
df_oc = df_2mu.Filter("Muon_charge[0] != Muon_charge[1]", "Muons with opposite charge")

df_mass = df_oc.Define("Dimuon_mass", "ROOT::VecOps::InvariantMass(Muon_pt, Muon_eta, Muon_phi, Muon_mass)")

nbins = 30000
low = 0.25
up = 300
histo_name = "Dimuon_mass"
histo_title = histo_name

h = df_mass.Histo1D((histo_name, histo_title, nbins, low, up), "Dimuon_mass")

ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetTextFont(42)
c = ROOT.TCanvas("c", "", 800, 700)
c.SetLogx()
c.SetLogy()
h.SetTitle("")
h.GetXaxis().SetTitle("m_{#mu#mu} (GeV)")
h.GetXaxis().SetTitleSize(0.04)
h.GetYaxis().SetTitle("N_{Events}")
h.GetYaxis().SetTitleSize(0.04)
h.Draw()

label = ROOT.TLatex()
label.SetNDC(True)
label.SetTextSize(0.040)
label.DrawLatex(0.100, 0.920, "#bf{CMS Open Data}")
label.SetTextSize(0.030)
label.DrawLatex(0.500, 0.920, "#sqrt{s} = 8 TeV, L_{int} = 11.6 fb^{-1}")

CPU times: user 825 ms, sys: 40 ms, total: 865 ms
Wall time: 868 ms


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


In [29]:
%jsroot

c.Draw()

In [30]:
#from dask.distributed import Client

#client = Client("localhost:23932")
#client

In [31]:
%%time

treename = "Events"
filename = "RNTuple_TEST_Run2012BC_DoubleMuParked_Muons.root"
df_rnt = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame(treename, filename, npartitions=4, daskclient=client)


# We need to register the header file for upload to the Dask workers.
# For now the interface is still WIP, will be made smoother in the next ROOT release
df_rnt._headnode.backend.distribute_unique_paths(
    ["rdataframe-dimuon.h"]
)

def my_initialization_function():
    """Load C++ helper functions. Works for both local and distributed execution."""
    try:
        # when using distributed RDataFrame 'rdataframe-dimuon.h' is copied to the local_directory
        # of every worker (via `distribute_unique_paths`)
        localdir = get_worker().local_directory
        cpp_header = Path(localdir) / "rdataframe-dimuon.h"
    except ValueError:
        # must be local execution
        cpp_header = "rdataframe-dimuon.h"

    ROOT.gInterpreter.Declare(f'#include "{str(cpp_header)}"')

ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)

CPU times: user 27.8 ms, sys: 1.78 ms, total: 29.6 ms
Wall time: 42 ms


In [32]:
%%time

df_2mu = df_rnt.Filter("nMuon == 2", "Events with exactly two muons")
df_oc = df_2mu.Filter("Muon_charge[0] != Muon_charge[1]", "Muons with opposite charge")

#df_mass = df_oc.Define("Dimuon_mass", "ROOT::VecOps::InvariantMass(Muon_pt, Muon_eta, Muon_phi, Muon_mass)")
df_mass_custom = df_oc.Define("Dimuon_mass_custom", "custom_InvariantMass(Muon_pt, Muon_eta, Muon_phi, Muon_mass)")

nbins = 30000
low = 0.25
up = 300
#histo_name = "Dimuon_mass"
#histo_title = histo_name

histo_name_custom = "Dimuon_mass_custom"
histo_title_custom = histo_name

#h = df_mass.Histo1D((histo_name, histo_title, nbins, low, up), "Dimuon_mass")
h_custom = df_mass_custom.Histo1D((histo_name_custom, histo_title_custom, nbins, low, up), "Dimuon_mass_custom")

ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetTextFont(42)
c_custom = ROOT.TCanvas("c_dask_rnt", "", 800, 700)
c_custom.SetLogx()
c_custom.SetLogy()
h_custom.SetTitle("")
h_custom.GetXaxis().SetTitle("m_{#mu#mu} (GeV)")
h_custom.GetXaxis().SetTitleSize(0.04)
h_custom.GetYaxis().SetTitle("N_{Events}")
h_custom.GetYaxis().SetTitleSize(0.04)
h_custom.Draw()

label = ROOT.TLatex()
label.SetNDC(True)
label.SetTextSize(0.040)
label.DrawLatex(0.100, 0.920, "#bf{CMS Open Data}")
label.SetTextSize(0.030)
label.DrawLatex(0.500, 0.920, "#sqrt{s} = 8 TeV, L_{int} = 11.6 fb^{-1}")

RuntimeError: C++ exception thrown:
	runtime_error: Cannot open 'd_Muons.root', error: No such file or directory

In [33]:
%jsroot

c_custom.Draw()